In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# # Install required libs
# !pip install git+https://github.com/qubvel/segmentation_models.pytorch
# !pip install albumentations==0.4.6

In [3]:
# !sudo rm -rf /content/seg_recyclables
# !git clone https://github.com/finani/seg_recyclables.git

# import sys
# sys.path.append('/content/seg_recyclables')

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

print('')

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

print('')

import os
import torch
print('Pytorch version: {}'.format(torch.__version__))
print('Is GPU available: {}'.format(torch.cuda.is_available()))
if torch.cuda.is_available():
  print(torch.cuda.get_device_name(0))
  print('The number of GPUs available: {}'.format(torch.cuda.device_count())) # Tesla P100-PCIE-16GB
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print('CPU count: {}'.format(os.cpu_count()))  # 8

cuda = torch.version.cuda
cudnn = torch.backends.cudnn.version()
cudnn_major = cudnn // 1000
cudnn = cudnn % 1000
cudnn_minor = cudnn // 100
cudnn_patch = cudnn % 100
print('Cuda version: {}'.format(cuda)) # 11.1
print('Cudnn version: {}.{}.{}'.format(cudnn_major, cudnn_minor, cudnn_patch)) # 8.0.5


Mon Dec 27 15:11:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| N/A   61C    P0    25W /  N/A |    441MiB /  7982MiB |     38%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Train Binary Segmentation
Classes =  ['Background', 'UNKNOWN', 'General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing']

## Configures

In [5]:
import os
import sys

import torch
import numpy as np

import segmentation_models_pytorch as smp
from torch.optim import lr_scheduler

import Utils
from ModelManager import ModelManager
from DataManager import DataManager, CustomAugmentation
from TrainManager import TrainManager
from InferManager import InferManager
from LossManager import DiceLoss
from LearningRateManager import CustomCosineAnnealingWarmUpRestarts

Utils.fix_random_seed(random_seed=21)

project_dir = '/home/weebee/recyclables/baseline'
dataset_dir = os.path.join(project_dir, 'input')
save_dir = os.path.join(project_dir, 'saved/test_1')
if not os.path.isdir(dataset_dir):
    sys.exit('check dataset path!!')
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)

batch_size = 5
number_worker = 4
epochs = 1
learning_rate = 1e-4

encoder = 'resnet101'
encoder_weights = 'imagenet'
activation = None
multi_gpu = False

transform = CustomAugmentation.to_tensor_transform()

In [6]:
# Make Model
model_manager = ModelManager()
model = model_manager.make_deeplabv3plus_model(
    encoder=encoder,
    encoder_weights=encoder_weights,
    class_number=2,
    activation=activation,
    multi_gpu=multi_gpu
)

criterion = smp.utils.losses.CrossEntropyLoss()
# criterion = smp.utils.losses.BCEWithLogitsLoss()
# criterion = tgm.losses.DiceLoss()
# criterion = DiceLoss()
# criterion = smp.utils.losses.JaccardLoss()

optimizer = torch.optim.Adam(
    [dict(params=model.parameters(),
          lr=learning_rate
          ),
     ])

lr_scheduler = None
# lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
#     optimizer, max_lr=0.01, steps_per_epoch=10, epochs=epochs, anneal_strategy='cos'
# )
# lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
#     optimizer, T_0=1, T_mult=2, eta_min=5e-5,
# )
# lr_scheduler = CustomCosineAnnealingWarmUpRestarts(
#     optimizer, T_0=20, T_mult=1, eta_max=0.1,  T_up=2, gamma=0.5
# )

data_manager = DataManager(dataset_path=dataset_dir)
train_manager = TrainManager()
infer_manager = InferManager()

print(Utils.get_classes())

## Train each class

In [ ]:
for target_class in Utils.get_classes()[1:]:
    # Set Target class
    target_classes = ['Background']
    target_classes.append(target_class)
    print('\n\ttarget_class: {}\n'.format(target_class))

    # Load Dataset
    data_manager.assignDataLoaders(
        batch_size=batch_size,
        shuffle=True,
        number_worker=number_worker,
        drop_last=False,
        transform=transform,
        target_segmentation=True,
        target_classes=target_classes
    )

    # Run Train
    model = train_manager.run_train(
        model=model,
        num_epochs=epochs,
        data_loader=data_manager.train_data_loader,
        val_loader=data_manager.val_data_loader,
        criterion=criterion,
        optimizer=optimizer,
        lr_scheduler=lr_scheduler,
        save_dir=save_dir,
        file_name='best_model_target_' + '_'.join([v.lower() for v in target_classes[1:]]) + '.pt',
        class_number=len(target_classes),
        val_every=1
    )

    # Check Trained Model
    infer_manager.run_predict(
        model=model,
        data_loader=data_manager.val_data_loader,
        data_index_list=range(0, batch_size, int(batch_size/5)),  # data_index < batch_size
        save_dir=save_dir,
        class_name_list=target_classes,
        mode='val'
    )


	target_class: UNKNOWN

loading annotations into memory...
Done (t=2.37s)
creating index...
index created!
loading annotations into memory...
Done (t=0.84s)
creating index...
index created!

Start training..
num_epochs: 1
save_dir: /home/weebee/recyclables/baseline/saved/test_1
file_name: best_model_target_unknown.pt
val_every: 1



Epoch: 0/1, Time: 2021-12-27 15:09:02, lr: 0.0001:  38%|███▊      | 200/524 [01:57<03:13,  1.68it/s, loss=0.0325, mean_loss=0.149]